# Import libraries and dependencies

In [1]:
# requirements.txt
!pip install pyarrow

In [2]:
import pandas as pd
import numpy as np
import os
import calendar
from datetime import datetime
import math

In [3]:
%ls data/

machina.tinyflux  sample.parquet


In [4]:
df = pd.read_parquet('data/sample.parquet', engine="pyarrow")

# 2.1 Preprocess and Clean

This is the EDA (aka exploratory data analysis) step...

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1546591 entries, 0 to 1546590
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   time         1546591 non-null  object 
 1   value        1546591 non-null  float64
 2   field        1546591 non-null  object 
 3   robot_id     1546591 non-null  int64  
 4   run_uuid     1546591 non-null  float64
 5   sensor_type  1546591 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 70.8+ MB


### notes: 

This is a preprocessing step below for python to display the full id rather than shortened form (ie. 3.380000e6 vs 3380000) by assigning it as an integer type and then as a string type.

In [6]:
# preprocessing
df['run_uuid'] = df['run_uuid'].astype(int).astype(str)
df['robot_id'] = df['robot_id'].astype(int).astype(str)
df['time'] = pd.to_datetime(df['time'])

In [7]:
df.sensor_type.unique()

array(['encoder', 'load_cell'], dtype=object)

In [8]:
print("min: ", df.time.min(), '\nmax: ', df.time.max())

min:  2022-11-23 20:40:00.001000+00:00 
max:  2022-11-23 20:49:59.999000+00:00


In [9]:
df.groupby('field').size()

field
fx    247764
fy    240000
fz    240000
x     300009
y     268810
z     250008
dtype: int64

In [10]:
df['value'].describe()

count    1.546591e+06
mean     3.531151e+02
std      9.832975e+02
min     -1.848419e+03
25%     -2.222900e+02
50%      1.723072e+02
75%      8.685362e+02
max      3.298350e+03
Name: value, dtype: float64

# 2.2 Convert timeseries to features by robot_id

preparing the data...

## expected headers
| time | fx_1 | fx_2 | fy_1 | fy_2 | fz_1 | fz_2 | x_1 | x_2 | y_1 | y_2 | z_1 | z_2 |

## decision point:
I decided not to pivot the field 'robot_id' to make it easier down the line as for processing and calculation efforts.

I see this as a benefit later downstream when we can consider 'robot_id' as a sharding attribute.  Pivoting 'robot_id' into column will actually make it use more resources (ie cpu and memory) as we have to write empty columns for each record between robot 1 and robot 2.  

## new fields
| time | run_uuid | sensor_type | robot_id | fx | fy | fz | x | y | z |

In [11]:
df2 = pd.pivot_table(df, values='value', index=['time', 'run_uuid', 'sensor_type', 'robot_id'], columns=['field'])

In [12]:
df2

field                                                                                fx  \
time                             run_uuid             sensor_type robot_id                
2022-11-23 20:40:00.001000+00:00 7582293080991469568  load_cell   1        -1192.046953   
2022-11-23 20:40:00.003000+00:00 -9223372036854775808 load_cell   1          -88.747061   
                                 7582293080991469568  load_cell   2         -546.669903   
2022-11-23 20:40:00.005000+00:00 8910095844186656768  encoder     1                 NaN   
2022-11-23 20:40:00.007000+00:00 6176976534744076288  load_cell   1          176.096381   
...                                                                                 ...   
2022-11-23 20:49:59.996000+00:00 7582293080991469568  encoder     2                 NaN   
2022-11-23 20:49:59.998000+00:00 7582293080991469568  load_cell   1        -1375.790361   
                                                                  2         -546.645184   
2022-11-23 20:49:59.999000+00:00 6176976534744076288  encoder     1                 NaN   
                                 8910095844186656768  encoder     2                 NaN   

field                                                                               fy  \
time                             run_uuid             sensor_type robot_id               
2022-11-23 20:40:00.001000+00:00 7582293080991469568  load_cell   1         716.528276   
2022-11-23 20:40:00.003000+00:00 -9223372036854775808 load_cell   1                NaN   
                                 7582293080991469568  load_cell   2         489.207227   
2022-11-23 20:40:00.005000+00:00 8910095844186656768  encoder     1                NaN   
2022-11-23 20:40:00.007000+00:00 6176976534744076288  load_cell   1         174.268623   
...                                                                                ...   
2022-11-23 20:49:59.996000+00:00 7582293080991469568  encoder     2                NaN   
2022-11-23 20:49:59.998000+00:00 7582293080991469568  load_cell   1        -208.161848   
                                                                  2         489.154858   
2022-11-23 20:49:59.999000+00:00 6176976534744076288  encoder     1                NaN   
                                 8910095844186656768  encoder     2                NaN   

field                                                                                fz  \
time                             run_uuid             sensor_type robot_id                
2022-11-23 20:40:00.001000+00:00 7582293080991469568  load_cell   1        -1547.340972   
2022-11-23 20:40:00.003000+00:00 -9223372036854775808 load_cell   1                 NaN   
                                 7582293080991469568  load_cell   2           84.484822   
2022-11-23 20:40:00.005000+00:00 8910095844186656768  encoder     1                 NaN   
2022-11-23 20:40:00.007000+00:00 6176976534744076288  load_cell   1         -258.179417   
...                                                                                 ...   
2022-11-23 20:49:59.996000+00:00 7582293080991469568  encoder     2                 NaN   
2022-11-23 20:49:59.998000+00:00 7582293080991469568  load_cell   1        -1388.172515   
                                                                  2           83.245564   
2022-11-23 20:49:59.999000+00:00 6176976534744076288  encoder     1                 NaN   
                                 8910095844186656768  encoder     2                 NaN   

field                                                                               x  \
time                             run_uuid             sensor_type robot_id              
2022-11-23 20:40:00.001000+00:00 7582293080991469568  load_cell   1               NaN   
2022-11-23 20:40:00.003000+00:00 -9223372036854775808 load_cell   1               NaN   
                                 7582293080991469568  load_cell   2               NaN   
2022-11-23 20:40

In [13]:
df2 = df2.fillna(0)

In [14]:
df2 = df2.sort_index(0, ascending=True)

/var/folders/vd/lfftpv6d31l986zrb_3v_7440000gn/T/ipykernel_13488/1153260469.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  df2 = df2.sort_index(0, ascending=True)


In [15]:
df3 = df2.reset_index()

In [16]:
df3

field,time,run_uuid,sensor_type,robot_id,fx,fy,fz,x,y,z
0,2022-11-23 20:40:00.001000+00:00,7582293080991469568,load_cell,1,-1192.046953,716.528276,-1547.340972,0.0000,0.0000,0.0000
1,2022-11-23 20:40:00.003000+00:00,-9223372036854775808,load_cell,1,-88.747061,0.000000,0.000000,0.0000,0.0000,0.0000
2,2022-11-23 20:40:00.003000+00:00,7582293080991469568,load_cell,2,-546.669903,489.207227,84.484822,0.0000,0.0000,0.0000
3,2022-11-23 20:40:00.005000+00:00,8910095844186656768,encoder,1,0.000000,0.000000,0.000000,821.7808,326.5256,-1.8051
4,2022-11-23 20:40:00.007000+00:00,6176976534744076288,load_cell,1,176.096381,174.268623,-258.179417,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...
547768,2022-11-23 20:49:59.996000+00:00,7582293080991469568,encoder,2,0.000000,0.000000,0.000000,3050.7730,1000.7690,-771.6320
547769,2022-11-23 20:49:59.998000+00:00,7582293080991469568,load_cell,1,-1375.790361,-208.161848,-1388.172515,0.0000,0.0000,0.0000
547770,2022-11-23 20:49:59.998000+00:00,7582293080991469568,load_cell,2,-546.645184,489.154858,83.245564,0.0000,0.0000,0.0000
547771,2022-11-23 20:49:59.999000+00:00,6176976534744076288,encoder,1,0.000000,0.000000,0.000000,1440.7900,936.9250,-222.2900


# 2.3 Include Engineered/Calculated Features

## feature generation
Need to calculate velocity, acceleration, total velocity, total acceleration, and total force
- 6 Velocity values (vx_1, vy_1, vz_1, vx_2, vy_2, vz_2)
- 6 Acceleration values (ax_1, ay_1, az_1, ax_2, ay_2, az_2)
- Total Velocity (v1, v2)
- Total Acceleration (a1, a2)
- Total Force (f1, f2)

In [17]:
# calculate total velocity, acceleration, force, distance
def calc_total(x, y, z):
    total = np.sqrt(x**2 + y**2 + z**2)
    return total

In [18]:
import sqlite3
conn = sqlite3.connect('machina_lab.db')

In [19]:
df2

field                                                                                fx  \
time                             run_uuid             sensor_type robot_id                
2022-11-23 20:40:00.001000+00:00 7582293080991469568  load_cell   1        -1192.046953   
2022-11-23 20:40:00.003000+00:00 -9223372036854775808 load_cell   1          -88.747061   
                                 7582293080991469568  load_cell   2         -546.669903   
2022-11-23 20:40:00.005000+00:00 8910095844186656768  encoder     1            0.000000   
2022-11-23 20:40:00.007000+00:00 6176976534744076288  load_cell   1          176.096381   
...                                                                                 ...   
2022-11-23 20:49:59.996000+00:00 7582293080991469568  encoder     2            0.000000   
2022-11-23 20:49:59.998000+00:00 7582293080991469568  load_cell   1        -1375.790361   
                                                                  2         -546.645184   
2022-11-23 20:49:59.999000+00:00 6176976534744076288  encoder     1            0.000000   
                                 8910095844186656768  encoder     2            0.000000   

field                                                                               fy  \
time                             run_uuid             sensor_type robot_id               
2022-11-23 20:40:00.001000+00:00 7582293080991469568  load_cell   1         716.528276   
2022-11-23 20:40:00.003000+00:00 -9223372036854775808 load_cell   1           0.000000   
                                 7582293080991469568  load_cell   2         489.207227   
2022-11-23 20:40:00.005000+00:00 8910095844186656768  encoder     1           0.000000   
2022-11-23 20:40:00.007000+00:00 6176976534744076288  load_cell   1         174.268623   
...                                                                                ...   
2022-11-23 20:49:59.996000+00:00 7582293080991469568  encoder     2           0.000000   
2022-11-23 20:49:59.998000+00:00 7582293080991469568  load_cell   1        -208.161848   
                                                                  2         489.154858   
2022-11-23 20:49:59.999000+00:00 6176976534744076288  encoder     1           0.000000   
                                 8910095844186656768  encoder     2           0.000000   

field                                                                                fz  \
time                             run_uuid             sensor_type robot_id                
2022-11-23 20:40:00.001000+00:00 7582293080991469568  load_cell   1        -1547.340972   
2022-11-23 20:40:00.003000+00:00 -9223372036854775808 load_cell   1            0.000000   
                                 7582293080991469568  load_cell   2           84.484822   
2022-11-23 20:40:00.005000+00:00 8910095844186656768  encoder     1            0.000000   
2022-11-23 20:40:00.007000+00:00 6176976534744076288  load_cell   1         -258.179417   
...                                                                                 ...   
2022-11-23 20:49:59.996000+00:00 7582293080991469568  encoder     2            0.000000   
2022-11-23 20:49:59.998000+00:00 7582293080991469568  load_cell   1        -1388.172515   
                                                                  2           83.245564   
2022-11-23 20:49:59.999000+00:00 6176976534744076288  encoder     1            0.000000   
                                 8910095844186656768  encoder     2            0.000000   

field                                                                               x  \
time                             run_uuid             sensor_type robot_id              
2022-11-23 20:40:00.001000+00:00 7582293080991469568  load_cell   1            0.0000   
2022-11-23 20:40:00.003000+00:00 -9223372036854775808 load_cell   1            0.0000   
                                 7582293080991469568  load_cell   2            0.0000   
2022-11-23 20:40

In [20]:
# init robot table
df3 = df2.reset_index()
idx_col = ['time', 'run_uuid', 'sensor_type', 'robot_id']
group_col = [pd.Grouper(key='time', freq='S'), 'run_uuid', 'sensor_type', 'robot_id']
df3.to_sql(name='robot', con=conn)
df3['total_force'] = calc_total(df3['fx'], df3['fy'], df3['fz'])
df3[idx_col+['total_force']].to_sql(name="force", con=conn)

# init displacement table
distance_df = df3[idx_col+['x', 'y', 'z']]
distance_df['xdiff'] = distance_df.groupby(group_col)['x'].diff()
distance_df['ydiff'] = distance_df.groupby(group_col)['y'].diff()
distance_df['zdiff'] = distance_df.groupby(group_col)['z'].diff()
distance_df['totaldistance'] = calc_total(distance_df['x'], distance_df['y'], distance_df['z'])
distance_df = distance_df[~(distance_df['xdiff'].isna())&
                          ~(distance_df['ydiff'].isna())&
                          ~(distance_df['zdiff'].isna())&
                          ~(distance_df['totaldistance'].isna())].drop(['x', 'y', 'z'], axis=1)
distance_df.to_sql(name='displacement', con=conn)

# init velocity table
velocity_df = distance_df
velocity_df['timediff'] = velocity_df.groupby(group_col)['time'].diff().dt.total_seconds()
velocity_df['velocityx'] = velocity_df['xdiff'] / velocity_df['timediff']
velocity_df['velocityy'] = velocity_df['ydiff'] / velocity_df['timediff']
velocity_df['velocityz'] = velocity_df['zdiff'] / velocity_df['timediff']
velocity_df['totalvelocity'] = calc_total(velocity_df['velocityx'], velocity_df['velocityy'], velocity_df['velocityz'])
velocity_df = velocity_df[~(velocity_df['velocityx'].isna())&
            ~(velocity_df['velocityy'].isna())&
            ~(velocity_df['velocityz'].isna())&
            ~(velocity_df['totalvelocity'].isna())&
            ~(velocity_df['timediff'].isna())].drop(['xdiff', 'ydiff', 'zdiff', 'totaldistance'], axis=1)
velocity_df.to_sql(name='velocity', con=conn)

# init acceleration table
acceleration_df = velocity_df
acceleration_df['accelx'] = acceleration_df.groupby(group_col)['velocityx'].diff()
acceleration_df['accely'] = acceleration_df.groupby(group_col)['velocityy'].diff()
acceleration_df['accelz'] = acceleration_df.groupby(group_col)['velocityz'].diff()
acceleration_df['totalacceleration'] = calc_total(acceleration_df['accelx'], acceleration_df['accely'], acceleration_df['accelz'])
acceleration_df = acceleration_df[~(acceleration_df['accelx'].isna())&
                ~(acceleration_df['accely'].isna())&
                ~(acceleration_df['accelz'].isna())&
                ~(acceleration_df['totalacceleration'].isna())]
acceleration_df.drop(['velocityx', 'velocityy', 'velocityz', 'totalvelocity'], axis=1).to_sql(name='acceleration', con=conn)


/var/folders/vd/lfftpv6d31l986zrb_3v_7440000gn/T/ipykernel_13488/1189205564.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance_df['xdiff'] = distance_df.groupby(group_col)['x'].diff()
/var/folders/vd/lfftpv6d31l986zrb_3v_7440000gn/T/ipykernel_13488/1189205564.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance_df['ydiff'] = distance_df.groupby(group_col)['y'].diff()
/var/folders/vd/lfftpv6d31l986zrb_3v_7440000gn/T/ipykernel_13488/1189205564.py:13: SettingWithCopyWarning: 
A value is try

529539

# 2.4 Calculate Runtime Statistics

In [21]:
df3[(df3['run_uuid']=='7582293080991469568')&(df3['sensor_type']=='encoder')&(df3['robot_id'])==1]

field,time,run_uuid,sensor_type,robot_id,fx,fy,fz,x,y,z,total_force
6,2022-11-23 20:40:00.008000+00:00,7582293080991469568,encoder,1,0.0,0.0,0.0,2862.302,995.860,-48.565,0.0
8,2022-11-23 20:40:00.009000+00:00,7582293080991469568,encoder,2,0.0,0.0,0.0,3050.773,1000.769,-771.632,0.0
18,2022-11-23 20:40:00.020000+00:00,7582293080991469568,encoder,1,0.0,0.0,0.0,2862.302,995.879,-48.568,0.0
20,2022-11-23 20:40:00.021000+00:00,7582293080991469568,encoder,2,0.0,0.0,0.0,3050.773,1000.769,-771.632,0.0
30,2022-11-23 20:40:00.032000+00:00,7582293080991469568,encoder,1,0.0,0.0,0.0,2862.302,995.899,-48.571,0.0
...,...,...,...,...,...,...,...,...,...,...,...
547746,2022-11-23 20:49:59.972000+00:00,7582293080991469568,encoder,2,0.0,0.0,0.0,3050.773,1000.769,-771.632,0.0
547755,2022-11-23 20:49:59.982000+00:00,7582293080991469568,encoder,1,0.0,0.0,0.0,3063.233,1225.812,-57.507,0.0
547757,2022-11-23 20:49:59.984000+00:00,7582293080991469568,encoder,2,0.0,0.0,0.0,3050.773,1000.769,-771.632,0.0
547766,2022-11-23 20:49:59.994000+00:00,7582293080991469568,encoder,1,0.0,0.0,0.0,3064.177,1225.877,-57.504,0.0


In [22]:
cur = conn.cursor()

In [23]:
cur.execute(
    '''
    select 
        run_uuid, 
        start_time, 
        stop_time, 
        round((julianday(stop_time)-julianday(start_time))*86400)/60 as run_time, 
        (max_distance-min_distance) as distance_traveled
    from (
        select 
            distinct r.run_uuid, 
            min(datetime(r.time)) over(partition by r.run_uuid) as start_time, 
            max(datetime(r.time)) over(partition by r.run_uuid) as stop_time,
            min(d.totaldistance) over(partition by r.run_uuid) as min_distance,
            max(d.totaldistance) over(partition by r.run_uuid) as max_distance
        from robot r 
        left join displacement d 
        on d.time=r.time and d.run_uuid=r.run_uuid and d.sensor_type=r.sensor_type and d.robot_id=r.robot_id
    ) s1
    ''')
runtime = cur.fetchall()
for el in runtime:
    print("run_uuid:", el[0])
    print("start_time:", el[1])
    print("stop_time:", el[2])
    print("run_time (min):", el[3])
    print("distance_traveled: ", el[4])
    print('='*50)

run_uuid: -9223372036854775808
start_time: 2022-11-23 20:40:00
stop_time: 2022-11-23 20:41:17
run_time (min): 1.2833333333333334
distance_traveled:  0.0
run_uuid: 6176976534744076288
start_time: 2022-11-23 20:40:00
stop_time: 2022-11-23 20:49:59
run_time (min): 9.983333333333333
distance_traveled:  1732.95178215004
run_uuid: 7582293080991469568
start_time: 2022-11-23 20:40:00
stop_time: 2022-11-23 20:49:59
run_time (min): 9.983333333333333
distance_traveled:  3461.6438194558664
run_uuid: 8910095844186656768
start_time: 2022-11-23 20:40:00
stop_time: 2022-11-23 20:49:59
run_time (min): 9.983333333333333
distance_traveled:  193.69070079412165


# 2.5 Store and Provide Access Tools

Below are the tables and schema created for this database: machina_lab.db.

I chose to build the database in a relational database due to the limited scope of the data and exercise.  Given more time, I would choose to build a time series database (ie such as influxDB, Apache Druid, etc) using MQTT protocols.  There are some advantages to going with a time series database for more efficient processing and ease of working with datetime data structure.  

Along with choosing a time series database, I would also shard the data across run_uuid, sensor_type and robot_id.  This will optimize the database tables.  
```
CREATE TABLE IF NOT EXISTS "robot" (
"index" INTEGER,
  "time" TIMESTAMP,
  "run_uuid" TEXT,
  "sensor_type" TEXT,
  "robot_id" TEXT,
  "fx" REAL,
  "fy" REAL,
  "fz" REAL,
  "x" REAL,
  "y" REAL,
  "z" REAL
);
CREATE INDEX "ix_robot_index"ON "robot" ("index");
CREATE TABLE IF NOT EXISTS "force" (
"index" INTEGER,
  "time" TIMESTAMP,
  "run_uuid" TEXT,
  "sensor_type" TEXT,
  "robot_id" TEXT,
  "total_force" REAL
);
CREATE INDEX "ix_force_index"ON "force" ("index");
CREATE TABLE IF NOT EXISTS "displacement" (
"index" INTEGER,
  "time" TIMESTAMP,
  "run_uuid" TEXT,
  "sensor_type" TEXT,
  "robot_id" TEXT,
  "xdiff" REAL,
  "ydiff" REAL,
  "zdiff" REAL,
  "totaldistance" REAL
);
CREATE INDEX "ix_displacement_index"ON "displacement" ("index");
CREATE TABLE IF NOT EXISTS "velocity" (
"index" INTEGER,
  "time" TIMESTAMP,
  "run_uuid" TEXT,
  "sensor_type" TEXT,
  "robot_id" TEXT,
  "timediff" REAL,
  "velocityx" REAL,
  "velocityy" REAL,
  "velocityz" REAL,
  "totalvelocity" REAL
);
CREATE INDEX "ix_velocity_index"ON "velocity" ("index");
CREATE TABLE IF NOT EXISTS "acceleration" (
"index" INTEGER,
  "time" TIMESTAMP,
  "run_uuid" TEXT,
  "sensor_type" TEXT,
  "robot_id" TEXT,
  "timediff" REAL,
  "accelx" REAL,
  "accely" REAL,
  "accelz" REAL,
  "totalacceleration" REAL
);
CREATE INDEX "ix_acceleration_index"ON "acceleration" ("index");
```